In [1]:
import pandas as pd
import pyodbc
import useful_functions as use
from functools import reduce

In [39]:
august_correct = pd.read_excel('../../Data/ValidationSurvey/VALIDATIONS_RETURN_090121.xlsx', sheet_name='Correct')
august_incorrect = pd.read_excel('../../Data/ValidationSurvey/VALIDATIONS_RETURN_090121.xlsx', sheet_name='Incorrect')
august_not = pd.read_excel('../../Data/ValidationSurvey/VALIDATIONS_RETURN_090121.xlsx', sheet_name='NotValidated')
august_un = pd.read_excel('../../Data/ValidationSurvey/VALIDATIONS_RETURN_090121.xlsx', sheet_name='Unfinalized')
survey = pd.concat([august_correct, august_incorrect, august_not, august_un])
survey['ME'] = use.fix_me(survey.ME_NO)
survey['IQVIA_ME'] = [x[0:10] for x in survey.ME]
survey['ORIGINAL_PHONE'] = [use.fix_phone(x) for x in survey.ORIGINAL_PHONE]

In [6]:
#dhc
dhc = pd.read_csv('../../Data/DHC/DHC_2021-11.csv')
dhc = dhc.fillna('None')
dhc['ME'] = use.fix_me(dhc.ME)
dhc['DHC_PHONE'] = [use.fix_phone(x) for x in dhc['Phone_Number']]

In [73]:
#data.gov
gov = pd.read_csv('../../Data/DataGov/All_Data_2021-11-09.csv')
gov = gov.fillna('None')
gov['ME'] = use.fix_me(gov.ME)
gov['GOV_PHONE'] = [use.fix_phone(x) for x in gov['phn_numbr']]

In [7]:
#iqvia
username = 'vigrose'
password = 'Ravenclaw~10946'
q = "DSN=eprdods; UID={}; PWD={}".format(username, password)
ODS = pyodbc.connect(q)
iqvia_query = \
        """
        SELECT DISTINCT 
        B.PHONE,
        P.ME,
        T.AFFIL_TYPE_DESC,
        A.AFFIL_IND,
        A.AFFIL_RANK
        FROM 
        ODS.ODS_IMS_BUSINESS B, ODS.SAS_ODS_IMS_PROVIDER_AFFIL A, ODS.ODS_IMS_PROFESSIONAL P, ODS.ODS_IMS_AFFILIATION_TYPE T
        WHERE  
        B.IMS_ORG_ID = A.IMS_ORG_ID
        AND
        A.PROFESSIONAL_ID = P.PROFESSIONAL_ID
        AND
        A.AFFIL_TYPE_ID = T.AFFIL_TYPE_ID
        AND
        P.CURRENT_BATCH_FLAG='Y'
        AND
        A.CURRENT_BATCH_FLAG='Y'
        AND
        B.CURRENT_BATCH_FLAG='Y'
        """
iqvia = pd.read_sql(con=ODS, sql=iqvia_query)
iqvia.head()

,PHONE,ME,AFFIL_TYPE_DESC,AFFIL_IND,AFFIL_RANK
0,2519493400,0010200001,Admitting,A,6
1,7275885200,0010200001,Admitting,A,6
2,8282131111,0010200003,Admitting,A,6
3,6235244000,0010200004,Attending,P,10
4,5048423000,0010200005,IDN Affiliated (Inferred),P,9


In [8]:
iqvia =iqvia.fillna('None')
iqvia['IQVIA_PHONE'] = [use.fix_phone(x) for x in iqvia['PHONE']]

In [9]:
#symphony
sym_query = \
        """
        SELECT
        d.ADDR_FRST_TLPHN_NBR AS TELEPHONE,
        l.OTHER_ID AS SYM_ME
        FROM
        ODS.PRACTITIONER_DEMOGRAPHIC_LAYOUT d, ODS.PRACTITIONER_ADDL_IDS_LAYOUT l
        WHERE
        d.DS_PRCTR_ID = l.DS_PRCTR_ID
        and
        l.ID_QLFR_TYP_CDE = 38
        """  
symphony = pd.read_sql(con=ODS, sql=sym_query)
symphony.head()

,TELEPHONE,SYM_ME
0,(979) 595-1700,7040284108
1,(214) 857-0273,4950993016
2,(409) 383-1355,4951288002
3,(740) 450-9200,4951486004
4,(813) 633-1100,4951688018


In [10]:
symphony =symphony.fillna('None')
symphony['SYM_PHONE'] = [use.fix_phone(x) for x in symphony['TELEPHONE']]

In [74]:
THIS = pd.merge(survey, symphony, left_on='IQVIA_ME', right_on='SYM_ME', how='left')
THIS = pd.merge(THIS, iqvia, left_on='IQVIA_ME', right_on='ME', how='left', suffixes = ['','_iqvia'])
THIS = pd.merge(THIS, dhc, on='ME', how='left')
THIS = pd.merge(THIS, gov, on='ME', how='left')[['ME','IQVIA_ME', 'ORIGINAL_PHONE','SYM_PHONE','IQVIA_PHONE', 'DHC_PHONE', 'GOV_PHONE', 'CORRECT_PHONE','REASON_PHONE_INCORRECT','REASON_PHONE_OTHER','CAPTURED_NUMBER','RESULT_OF_CALL']].drop_duplicates()
THIS = THIS.fillna('None')

In [118]:
dict_list = []
for row in THIS.itertuples():
    count = 0
    MATCHES = ''
    phone = row.ORIGINAL_PHONE
    symph_phone = row.SYM_PHONE
    dhc_phone = row.DHC_PHONE
    iqvia_phone = row.IQVIA_PHONE
    data_phone = row.GOV_PHONE
    total = 4-[symph_phone, dhc_phone, iqvia_phone, data_phone].count('None')
    if symph_phone == phone:
        count += 1
        MATCHES += 'Symphony, '
    if dhc_phone == phone:
        count += 1
        MATCHES += 'DHC, '
    if iqvia_phone == phone:
        count += 1
        MATCHES += 'IQVia, '
    if data_phone == phone:
        count += 1
        MATCHES += 'DataGov, '
    if total!=0:
        score = count/total
    else:
        score = 'None'
    if phone!='None':
        dicto = {
        'ME': row.ME,
        'IQVIA_ME': row.IQVIA_ME,
        'PHONE_SURVEY': phone,
        'CORRECT': row.CORRECT_PHONE,
        'CAPTURED_NUMBER':row.CAPTURED_NUMBER,
        'RESULT_OF_CALL':row.RESULT_OF_CALL,
        'MATCHED': count,
        'MATCHES': MATCHES[:-2],
        'REASON_PHONE_INCORRECT':row.REASON_PHONE_INCORRECT,
        'REASON_PHONE_OTHER':row.REASON_PHONE_OTHER,
        'TOTAL':total,
        'SCORE': score
    }
        dict_list.append(dicto)

In [119]:
phone_matches = pd.DataFrame(dict_list)

In [120]:
phone_matches = phone_matches.sort_values('MATCHED', ascending=False).drop_duplicates('ME')

In [105]:
phone_matches.groupby(['RESULT_OF_CALL']).mean()[['MATCHED']]

,MATCHED
RESULT_OF_CALL,
Changed Number,0.500000
Complete,1.239609
Contact Too Busy,0.888889
Deceased,0.500000
Direct No,1.000000
Disconnect,0.742647
Do Not Call,1.000000
Office Closed Message,0.937500
Other No Contact Reason,0.925993


In [61]:
phone_matches.groupby(['MATCHES','RESULT_OF_CALL']).count()[['ME']]

ME
MATCHES                  RESULT_OF_CALL       
                         Changed Number      6
                         Complete          379
                         Contact Too Busy    5
                         Deceased            1
                         Direct No           7
...                                        ...
Symphony, IQVia          Ring No Answer      1
Symphony, IQVia, DataGov Complete            3
                         Contact Too Busy    1
                         Direct No           1
                         Disconnect          1

[72 rows x 1 columns]

In [79]:
with pd.ExcelWriter(f'../../Data/ValidationSurvey/August_Triangulation.xlsx') as writer:  
        phone_matches.to_excel(writer, sheet_name='Results')
        phone_matches.groupby('MATCHES').count()[['ME']].to_excel(writer, sheet_name='All Matches')
        phone_matches.groupby('MATCHED').count()[['ME']].to_excel(writer, sheet_name='Match Count')
        phone_matches.groupby(['MATCHES','RESULT_OF_CALL']).count()[['ME']].to_excel(writer, sheet_name='Matches and Result of Call')
        phone_matches.groupby(['MATCHES','CORRECT']).count()[['ME']].to_excel(writer, sheet_name='Matches and Correctness')
        phone_matches.groupby(['RESULT_OF_CALL','MATCHED']).count()[['ME']].to_excel(writer, sheet_name='Result of Call - Matched')
        phone_matches.groupby(['CORRECT','MATCHED']).count()[['ME']].to_excel(writer, sheet_name='Correctness - Matched')
        phone_matches.groupby(['CORRECT']).mean()[['MATCHED']].to_excel(writer, sheet_name='Average # of Matches')

In [117]:
phone_matches.to_csv('../../Data/ValidationSurvey/August_eek.csv', index=False)

In [121]:
phone_matches[phone_matches.SCORE!='None'].groupby('CORRECT').mean()

,MATCHED,TOTAL
CORRECT,,
N,0.550868,3.069479
None,0.858252,3.218447
Y,1.568510,3.508413


In [122]:
boop = phone_matches[phone_matches.SCORE!='None']
boop['SCORE'] = boop.SCORE.astype(float)
boop.groupby('CORRECT').mean()

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,MATCHED,TOTAL,SCORE
CORRECT,,,
N,0.550868,3.069479,0.193135
None,0.858252,3.218447,0.269903
Y,1.568510,3.508413,0.452324


In [127]:
phone_matches.groupby(['SCORE']).count()[['ME']]

,ME
SCORE,
0.0,888
0.25,265
0.3333333333333333,213
0.5,388
0.6666666666666666,190
0.75,187
1.0,134
None,4
